In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import glob

In [2]:
# Read in one csv file to have an overview of the data
df_201801 = pd.read_csv("data/201801-citibike-tripdata.csv")
df_201801.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,970,2018-01-01 13:50:57.4340,2018-01-01 14:07:08.1860,72,W 52 St & 11 Ave,40.767272,-73.993929,505,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1
1,723,2018-01-01 15:33:30.1820,2018-01-01 15:45:33.3410,72,W 52 St & 11 Ave,40.767272,-73.993929,3255,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1
2,496,2018-01-01 15:39:18.3370,2018-01-01 15:47:35.1720,72,W 52 St & 11 Ave,40.767272,-73.993929,525,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1
3,306,2018-01-01 15:40:13.3720,2018-01-01 15:45:20.1910,72,W 52 St & 11 Ave,40.767272,-73.993929,447,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1
4,306,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72,W 52 St & 11 Ave,40.767272,-73.993929,3356,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1


In [3]:
# Unique values in the usertype column
df_201801.usertype.unique()

array(['Subscriber', 'Customer'], dtype=object)

In [4]:
# Unique values in the gender column
df_201801.gender.unique()

array([1, 0, 2], dtype=int64)

As explained in the [citi bike data website](https://www.citibikenyc.com/system-data):
* Trip Duration (seconds)
* User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
* Gender (Zero=unknown; 1=male; 2=female)

Hence, we would need to translate trip duration from seconds to minutes, also translate gender from numerical categories to word categories. It also makes more sense to include only trips that last more than 30 minutes.

In [5]:
# Get a sample of the dataframe to preprocess the data
df_201801_sampled = df_201801.sample(10, random_state=1)
df_201801_sampled

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
493184,433,2018-01-24 09:18:09.9380,2018-01-24 09:25:23.5580,3116,Huron St & Franklin St,40.732660,-73.958260,3092,Berry St & N 8 St,40.719009,-73.958525,26373,Subscriber,1988,1
514807,385,2018-01-25 11:00:12.3490,2018-01-25 11:06:37.7620,3147,E 85 St & 3 Ave,40.778012,-73.954071,3378,E 76 St & Park Ave,40.773763,-73.962221,20067,Subscriber,1962,2
570028,3982,2018-01-21 08:49:01.7740,2018-01-21 09:55:24.7110,3256,Pier 40 - Hudson River Park,40.727714,-74.011296,519,Pershing Square North,40.751873,-73.977706,33068,Customer,1981,1
650742,443,2018-01-30 13:25:20.5700,2018-01-30 13:32:44.4960,3430,Richardson St & N Henry St,40.719079,-73.942237,3430,Richardson St & N Henry St,40.719079,-73.942237,19504,Subscriber,1962,1
526225,82,2018-01-20 23:01:04.1690,2018-01-20 23:02:26.3990,3161,W 76 St & Columbus Ave,40.780184,-73.977285,3160,Central Park West & W 76 St,40.778968,-73.973747,14919,Subscriber,1960,1
646415,305,2018-01-12 23:20:18.4350,2018-01-12 23:25:24.2220,3427,Lafayette St & Jersey St,40.724305,-73.996010,531,Forsyth St & Broome St,40.718939,-73.992663,18256,Subscriber,1956,1
435334,1069,2018-01-09 18:46:12.0840,2018-01-09 19:04:01.5320,537,Lexington Ave & E 24 St,40.740259,-73.984092,410,Suffolk St & Stanton St,40.720664,-73.985180,17365,Subscriber,1991,1
286307,1260,2018-01-03 09:32:03.9800,2018-01-03 09:53:04.1130,461,E 20 St & 2 Ave,40.735877,-73.982050,3461,Murray St & Greenwich St,40.714852,-74.011223,18003,Subscriber,1990,1
195250,2725,2018-01-22 12:47:16.1420,2018-01-22 13:32:42.1130,387,Centre St & Chambers St,40.712733,-74.004607,217,Old Fulton St,40.702772,-73.993836,18004,Customer,1969,0
233757,337,2018-01-27 18:05:42.0910,2018-01-27 18:11:19.8640,426,West St & Chambers St,40.717548,-74.013221,347,Greenwich St & W Houston St,40.728846,-74.008591,27214,Subscriber,1994,1


In [6]:
# Define a function to preprocess data
min_minutes = 30

def preprocess(df):
    
    #Translate trip duration from seconds to minutes
    df.iloc[:,0] = round(df.iloc[:,0]/60,0).astype(int)
    df = df[df.iloc[:, 0] >= min_minutes].copy()
    
    # Replace gender in numerical categories to word categories
    df.iloc[:,-1].replace(to_replace=[0, 1, 2],
                          value=['Unknown', 'Male', 'Female'],
                          inplace=True)
    df.drop_duplicates(inplace=True)
    
    return df

In [7]:
preprocess(df_201801_sampled)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
570028,66,2018-01-21 08:49:01.7740,2018-01-21 09:55:24.7110,3256,Pier 40 - Hudson River Park,40.727714,-74.011296,519,Pershing Square North,40.751873,-73.977706,33068,Customer,1981,Male
195250,45,2018-01-22 12:47:16.1420,2018-01-22 13:32:42.1130,387,Centre St & Chambers St,40.712733,-74.004607,217,Old Fulton St,40.702772,-73.993836,18004,Customer,1969,Unknown


In [8]:
# Define the path that includes the csv files to merge
files_to_merge = [file for file in glob.glob(r'data/*.csv')]

In [9]:
# Check the number of columns in each file
number_of_cols = list()

for file in files_to_merge:
    df = pd.read_csv(file, nrows = 0)
    number_of_cols.append(df.columns.size)
    
print(f"Maximum number of columns: {max(number_of_cols)}")
print(f"Minimum number of columns: {min(number_of_cols)}")

Maximum number of columns: 15
Minimum number of columns: 15


In [10]:
# Check if each file has column names
col_names = pd.concat([pd.read_csv(file, nrows = 0) for file in files_to_merge], ignore_index = True).columns
col_names

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

The column names are consistent across files.

In [11]:
CHUNK_SIZE = 200000

first_file = True

for file in files_to_merge:

    if first_file:
        df_chunk = pd.read_csv(file, chunksize=CHUNK_SIZE)
        
    # if it is not the first csv file then skip the header row (row 0) of that file
    else:
        skip_row = [0]
        df_chunk = pd.read_csv(file, chunksize=CHUNK_SIZE, skiprows=skip_row, header=None)

    # The above operation results in a TextFileReader object for iteration.
    # df_chunk is not a dataframe but an object for further operation in the next step.

    for chunk in df_chunk:
        # Preprocess each chunk
        processed_chunk = preprocess(chunk)
        
        # Append each chunk to a csv file 
        processed_chunk.to_csv("data/conso_file.csv", mode="a", index=False)
        
    first_file = False